In [1]:
import gc
from glob import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import yaml

import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics

import optuna
from optuna.visualization import (
    plot_contour
    , plot_edf
    , plot_intermediate_values
    , plot_optimization_history
    , plot_parallel_coordinate
    , plot_param_importances
    , plot_slice
)

import sys
sys.path.append("../utils")
from metrics import compute_recall_at4, compute_normalized_gini, compute_amex_metric

np.random.seed(2112)
pd.set_option('display.max_columns', None)

In [2]:
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "jupyterlab"

In [3]:
lgb.__version__

'3.3.2'

***
## load and prepare data

In [4]:
!ls ../data/processed/dsv04

test.parquet  train.parquet


In [5]:
train = pd.read_parquet("../data/processed/dsv04/train.parquet")
train_labels = pd.read_csv("../data/raw/train_labels.csv", index_col="customer_ID")

In [6]:
input_feats = train.columns.tolist()
len(input_feats)

1639

In [7]:
train = pd.merge(train, train_labels, how="inner", left_index=True, right_index=True)
train = train.reset_index()

del train_labels
gc.collect()

0

***
## model tuning

train with repeated cross validation

In [8]:
skf = StratifiedKFold(n_splits=3, random_state=2112, shuffle=True)
skf_split = list(skf.split(train, train["target"].values))

In [9]:
with open("../data/config/lgbm-bce-dsv04.yml", "r") as file:
    default_params = yaml.safe_load(file)
    file.close()

del default_params["num_iterations"]
default_params["boosting"] = "dart"

display(default_params)

{'bagging_fraction': 1.0,
 'bagging_freq': 1,
 'bin_construct_sample_cnt': 100000000,
 'feature_fraction': 0.15000000000000002,
 'feature_pre_filter': True,
 'force_col_wise': True,
 'lambda_l1': 5.996099571922015,
 'lambda_l2': 2.8900783163910697,
 'learning_rate': 0.05,
 'max_bin': 63,
 'metric': 'None',
 'min_data_in_leaf': 1000,
 'min_gain_to_split': 0.313937968985787,
 'num_leaves': 15,
 'objective': 'binary',
 'path_smooth': 1.476306537276899,
 'seed': 2112,
 'verbosity': -1,
 'boosting': 'dart'}

In [10]:
def train_models(dataframe: pd.DataFrame, split: list, model_params: dict) -> pd.DataFrame:
    
    # dataframe to store the oof predictions
    oof = dataframe[["target"]].copy()
    oof["pred"] = -1

    for train_idx,valid_idx in split:
        
        train_df = dataframe.loc[train_idx,:]
        valid_df = dataframe.loc[valid_idx,:]
        
        train_dset = lgb.Dataset(
            data=train_df[input_feats],
            label=train_df["target"].values,
            free_raw_data=True
        )        
        model = lgb.train(
            params=model_params,
            train_set=train_dset
        )        
        oof.loc[valid_idx,"pred"] = model.predict(valid_df[input_feats])
            
        del train_dset,model
        gc.collect()
    
    return oof

In [11]:
def objective(trial):
    sampled_params = dict(
        num_iterations = trial.suggest_int("num_iterations", 1000, 7000, 50),
        # dart
        drop_rate = trial.suggest_discrete_uniform("drop_rate", 0.05, 0.3, 0.01),
        max_drop = trial.suggest_int("max_drop", 10, 500, 10),
        skip_drop = trial.suggest_discrete_uniform("skip_drop", 0.2, 0.8, 0.05),        
    )
    model_params = {**default_params, **sampled_params}
    
    oof = train_models(train, skf_split, model_params)
    metric = compute_amex_metric(oof.target.values, oof.pred.values)
    return metric

In [12]:
do_optimize = True

study = optuna.create_study(
    study_name="lgbm-dart-bce-dsv04",
    direction='maximize',
    storage='sqlite:///lgbm-dart-bce-dsv04.db',
    load_if_exists=True,
)

if do_optimize:
    study.optimize(
        objective, 
        n_trials=1000, 
        timeout=345600, #4-days
        n_jobs=1, 
        gc_after_trial=True,
    ) 

[I 2022-08-04 03:41:47,137] Using an existing study with name 'lgbm-dart-bce-dsv04' instead of creating a new one.


/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:177: UserWarning:

Found `num_iterations` in params. Will use it instead of argument



[I 2022-08-04 04:19:02,155] Trial 68 finished with value: 0.7957112516672616 and parameters: {'num_iterations': 2750, 'drop_rate': 0.060000000000000005, 'max_drop': 120, 'skip_drop': 0.65}. Best is trial 62 with value: 0.7968637138993222.


/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:177: UserWarning:

Found `num_iterations` in params. Will use it instead of argument



[I 2022-08-04 05:07:05,040] Trial 69 finished with value: 0.7964764858825374 and parameters: {'num_iterations': 4300, 'drop_rate': 0.09, 'max_drop': 100, 'skip_drop': 0.75}. Best is trial 62 with value: 0.7968637138993222.


...

[I 2022-08-08 02:36:08,665] Trial 148 finished with value: 0.7957706039391443 and parameters: {'num_iterations': 5900, 'drop_rate': 0.08, 'max_drop': 340, 'skip_drop': 0.45}. Best is trial 62 with value: 0.7968637138993222.


/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:177: UserWarning:

Found `num_iterations` in params. Will use it instead of argument



[I 2022-08-08 03:39:03,699] Trial 149 finished with value: 0.7962627545979319 and parameters: {'num_iterations': 5500, 'drop_rate': 0.14, 'max_drop': 70, 'skip_drop': 0.65}. Best is trial 62 with value: 0.7968637138993222.


/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:177: UserWarning:

Found `num_iterations` in params. Will use it instead of argument



[I 2022-08-08 04:48:05,968] Trial 150 finished with value: 0.7957108310503919 and parameters: {'num_iterations': 4450, 'drop_rate': 0.09, 'max_drop': 290, 'skip_drop': 0.75}. Best is trial 62 with value: 0.7968637138993222.


In [13]:
study.trials_dataframe().sort_values("value", ascending=False).head(20)

,number,value,datetime_start,datetime_complete,duration,params_drop_rate,params_max_drop,params_num_iterations,params_skip_drop,state
62,62,0.796864,2022-08-03 22:14:26.784423,2022-08-03 22:50:00.732739,0 days 00:35:33.948316,0.10,40,4350,0.70,COMPLETE
112,112,0.796681,2022-08-06 06:20:11.251245,2022-08-06 07:03:24.908742,0 days 00:43:13.657497,0.19,50,4950,0.70,COMPLETE
114,114,0.796663,2022-08-06 07:52:42.895430,2022-08-06 08:38:52.869665,0 days 00:46:09.974235,0.20,60,4300,0.65,COMPLETE
16,16,0.796638,2022-07-29 14:42:08.222255,2022-07-29 17:21:58.205647,0 days 02:39:49.983392,0.14,220,5750,0.60,COMPLETE
79,79,0.796573,2022-08-04 15:33:18.245685,2022-08-04 16:23:17.068902,0 days 00:49:58.823217,0.15,60,4650,0.65,COMPLETE
28,28,0.796567,2022-07-30 17:30:28.167280,2022-07-30 20:32:23.234022,0 days 03:01:55.066742,0.20,490,5250,0.70,COMPLETE
50,50,0.796478,2022-08-01 23:37:29.570490,2022-08-02 02:05:19.308352,0 days 02:27:49.737862,0.17,250,4800,0.60,COMPLETE
69,69,0.796476,2022-08-04 04:19:02.323358,2022-08-04 05:07:05.016649,0 days 00:48:02.693291,0.09,100,4300,0.75,COMPLETE
22,22,0.796458,2022-07-30 07:36:39.928330,2022-07-30 08:55:42.709861,0 days 01:19:02.781531,0.08,210,4250,0.65,COMPLETE
44,44,0.796451,2022-08-01 09:50:23.076169,2022-08-01 12:27:05.024984,0 days 02:36:41.948815,0.19,360,4750,0.65,COMPLETE


In [14]:
plot_optimization_history(study)

In [15]:
try:
    plot_param_importances(study)
except:
    pass

In [16]:
plot_slice(study)

In [17]:
plot_edf(study)

In [18]:
plot_parallel_coordinate(study)

In [19]:
best_params = dict(study.best_params)
best_params = {**default_params, **best_params}
best_params

{'bagging_fraction': 1.0,
 'bagging_freq': 1,
 'bin_construct_sample_cnt': 100000000,
 'feature_fraction': 0.15000000000000002,
 'feature_pre_filter': True,
 'force_col_wise': True,
 'lambda_l1': 5.996099571922015,
 'lambda_l2': 2.8900783163910697,
 'learning_rate': 0.05,
 'max_bin': 63,
 'metric': 'None',
 'min_data_in_leaf': 1000,
 'min_gain_to_split': 0.313937968985787,
 'num_leaves': 15,
 'objective': 'binary',
 'path_smooth': 1.476306537276899,
 'seed': 2112,
 'verbosity': -1,
 'boosting': 'dart',
 'drop_rate': 0.1,
 'max_drop': 40,
 'num_iterations': 4350,
 'skip_drop': 0.7}

In [20]:
with open("../data/config/lgbm-dart-bce-dsv04.yml", "w") as file:
    yaml.dump(best_params, file, default_flow_style=False)
    file.close()

***